In [ ]:
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8n-seg.pt')
# load a pretrained model (recommended for training)
# Train the model
results = model.train(data='coco8-seg.yaml', epochs=100, imgsz=640)

In [ ]:
# tracking with track mark
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

video_path = "../../00.Data/08.YOLO/people_walk.mp4"
cap = cv2.VideoCapture(video_path)
f = open('./segment_log.txt', 'w+')

track_history = defaultdict(lambda: [])

while cap.isOpened():

    success, frame = cap.read()
    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        f.write('boxes : ' + str(boxes))
        f.write('track_ids : ' + str(track_ids))

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) # x, y center point
            if len(track) > 30: # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))

            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 200), thickness=5)

            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

            # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

f.close()
cap.release()
cv2.destroyAllWindows()